In [ ]:
import tensorflow as tf
import time
import graph

#import models
%run -n models.ipynb

In [ ]:
flags = tf.app.flags
FLAGS = flags.FLAGS
# 0.1 for cnn2, 0.3 for fgcnn2
flags.DEFINE_float('learning_rate', 0.3, 'Initial learning rate.')
flags.DEFINE_integer('batch_size', 100, 'Batch size.')
flags.DEFINE_float('regularization', 0, 'L2 regularizations of weights and biases.')

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data_mnist", one_hot=False)

# Feature graph

In [ ]:
t_start = time.process_time()
A = graph.adjacency(graph.grid(28), k=4)

# Connections are only vertical or horizontal on the grid. Corner vertices are connected to 2 neightbors only.
if False:
    import scipy.sparse
    A = A.toarray()
    A[A < A.max()/1.5] = 0
    A = scipy.sparse.csr_matrix(A)
    print('{} edges'.format(A.nnz))

L, _ = graph.laplacian(A, True)
lamb, U = graph.fourier(L)
print('Spectrum in [{:1.2e}, {:1.2e}]'.format(lamb[0], lamb[-1]))
print('Execution time: {:.2f}s'.format(time.process_time() - t_start))

# Neural network

In [ ]:
#x = tf.placeholder(tf.float32, (FLAGS.batch_size, 784))
#y = tf.placeholder(tf.int32, (FLAGS.batch_size))
x = tf.placeholder(tf.float32, (None, 784))
y = tf.placeholder(tf.int32, (None))

model = fc1()
model = fc2(nhiddens=100)
model = cnn2(K=5, F=10)  # K=28 is equivalent to filtering with fgcnn.
model = fcnn2(F=10)
model = fgcnn2(U, F=10)

# Construct computational graph
logits = model.inference(x)
loss = model.loss(logits, y, FLAGS.regularization)
train_op = model.training(loss, FLAGS.learning_rate)
eval_correct = model.evaluation(logits, y)

# Train
t_start = time.process_time()
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)
for i in range(int(1e4)):
    batch_xs, batch_ys = mnist.train.next_batch(FLAGS.batch_size)
    sess.run(train_op, feed_dict={x: batch_xs, y: batch_ys})
print('Training time: {:.2f}s'.format(time.process_time() - t_start))

# Evaluate
ncorrects = sess.run(eval_correct, feed_dict={x: mnist.test.images, y: mnist.test.labels})
precision = ncorrects / mnist.test.num_examples
print('Precision: {:.2f}% ({:d} / {:d})'.format(precision*100, ncorrects, mnist.test.num_examples))
sess.close()